# Demonstrating the reusability of [importnb](https://github.com/deathbeds/importnb).

* `importnb` imports notebooks as modules
* `importnb` allows notebooks to be discovered as tests.

    >>> with Notebook(): import test_notebook as nb
    >>> assert 'test_function' in dir(nb)
    >>> assert nb.notebook_directory_to_df()

# This notebooks loads in a directory of notebooks as dataframe.

In [1]:
    from toolz.curried import *
    from toolz.curried.operator import *
    from pathlib import Path
    from pandas import Series, DataFrame, concat as Concat

    ignores = 'checkpoint',

    from nbformat.v4 import reads

    def notebook_directory_to_df(location=''):
        """Convert a directory of notebooks in dataframes.

        returns a dataframe for the cells and for the metadata.
        """
        cells = metadata = None

        suffix = '.ipynb'
        recursive = False
        for path in getattr(Path(location), 'glob')(f"*{recursive and 'r' or ''}{suffix}"):
            for ignore in ignores:
                if ignore in str(path): continue

            nb = pipe(path.read_text(), reads)

            s = DataFrame({k: v for k, v in nb.items() if k !='cells'})
            s = s.metadata.apply(Series).stack().append(Concat({'nbformat': s.iloc[0][['nbformat', 'nbformat_minor']]}))

            if cells is None:
                cells = Concat({path: DataFrame(nb['cells'])})
                metadata = Concat({path: s})
            else:
                cells = cells.append(Concat({path: DataFrame(nb['cells'])}))
                metadata = metadata.append(Concat({path: s}))
        return cells, metadata.unstack([-2, -1])

### [Computational Color](http://printingcode.runemadsen.com/lecture-color/)

In [2]:
    from importnb import Notebook

In [3]:
    __all__ = 'test_function',

In [4]:
    from poser import *

# ipywidget

In [5]:
from traitlets import dlink
from ipywidgets import Text, HTML

df = None

query = Text(description='Find cells containing a string.')
table = HTML()

def finder(query):
    global df
    if df is None:
        df, metadata = notebook_directory_to_df()
        globals()['df'] = df
    return df[df.source.apply(''.join).apply(lambda x: query in x)].to_html()
    
dlink((query, 'value'), (table, 'value'), finder);

In [6]:
widget = λ.ipywidgets.VBox(children=[query, table])()

In [7]:
    @λ.ipywidgets.interact()
    def widget_demo(a=10):
        ...

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), Output()), _dom_classes=('widget-…

## pytest will discover `test_function` because it starts with `test_*`

In [8]:
    def test_function():
        assert True

## doctest can test the current notebook state.

In [9]:
    if __name__ == '__main__':
        import doctest
        doctest.testmod()

## doctest can test the future state of the module.

In [10]:
    if __name__ == '__main__':
        with Notebook():
            import test_notebook as nb
            doctest.testmod(nb)

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), Output()), _dom_classes=('widget-…